<a href="https://colab.research.google.com/github/thanujTa/machine-learning-projects/blob/main/Natural_language_processing_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro- NLP Fundementals

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-04-19 07:12:19--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-04-19 07:12:19 (72.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get text dataset

the dataset we're going to be using is kaggle's intro to NLP Dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data
unzip_data("nlp_getting_started.zip")

--2022-04-19 07:14:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.128, 74.125.69.128, 64.233.183.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-04-19 07:14:56 (104 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualise the Data

In [8]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [15]:
# how many examples of each classs are there
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [17]:
# how many total samples
len(train_df)


7613

In [18]:
len(test_df)

3263

In [23]:
# lets visualise some random training samples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text, target=row
  print(f"target:{target}"," real disaster" if target > 0 else"(not real disaster)")
  print(f"text:\n{text}\n")
  print("---\n")

target:0 (not real disaster)
text:
Last Second Ebay Bid RT? http://t.co/oEKUcq4ZL0 Shaolin Rescuers (dvd 2010) Shen Chan Nan Chiang Five Venoms Kung Fu ?Please Favori

---

target:1  real disaster
text:
A brief violent storm swept through the Chicago area Sunday afternoon leading to one death and an evacuation of Lollapalooza and more

---

target:0 (not real disaster)
text:
Owner of Chicago-Area Gay Bar Admits to Arson Scheme: Frank Elliott pleaded guilty to hiring an arsonist to to... http://t.co/jCFEhrHLq8

---

target:1  real disaster
text:
Senators calling for emergency housing: Boxer Feinstein back plan to move #homeless vets to VA campus http://t.co/Gm80X3vutf

---

target:0 (not real disaster)
text:
@DmoneyDemi I had my meltdown yesterday.  I'm going to miss you so much. You are forsure my DTB for life. When I get back watchout ??

---



### Split data into train and validation set

In [24]:
from sklearn.model_selection import train_test_split

In [27]:
# Use train_test split to split training data to train and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, random_state = 42)

In [29]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [30]:
len(train_df_shuffled)

7613

In [32]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting Text into numbers

### Text vectorization (Tokenization)

In [33]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [38]:
from tensorflow.keras.layers import TextVectorization

# use default TextVectorization parameters

text_vectorizer =  TextVectorization(max_tokens =None,
                                     standardize = "lower_and_strip_punctuation",
                                     split = "whitespace",
                                     ngrams= None,
                                     output_mode = "int",
                                     output_sequence_length = None)

In [39]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [42]:
# find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/ len(train_sentences))

15

In [46]:
# setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in vocabulary
max_length = 15 # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens =max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [47]:
#fit the text vectorizer
text_vectorizer.adapt(train_sentences)

In [48]:
#create a sample sentence and tokenize it
sample = "there's a flood in my street!"
text_vectorizer([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [50]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"original text:\n{random_sentence}\n\n\nvectorized version:")
text_vectorizer([random_sentence])

original text:
'Go too Ibiza Pop ah Pill Get DRUNK &amp; Fall off a Cliff'. (Real Talk) @DannyJohnJules @Spencer_Fearon @ChristieLinford


vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 112,  150,    1, 1796, 1907, 4917,   52, 1301,   35,  278,  102,
           3,  403,  369,  833]])>

In [51]:
# get the unique words in the vocabulary
words = text_vectorizer.get_vocabulary()# get all of the unique words in data
top_5_words = words[:5]
bottom_5_words = words[-5:]
print(f"5 most common words:{top_5_words}")
print(f"5 least used words:{bottom_5_words}")

5 most common words:['', '[UNK]', 'the', 'a', 'in']
5 least used words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Create an Embedding using Embedding Layer

to make our embeddind we use embedding layer in tensorflow.

the parameters we care most about for our embedding layer:
*`input_dim`
*`output_dim`
*`input_length`

In [53]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length)

embedding 

In [66]:
# get a sample sentence
random_sentence  = random.choice(train_sentences)
print(f"original text:{random_sentence}\
          \n\n Embedded version:")

# embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

original text:An IS group suicide bomber detonated an explosives-packed vest in a mosque inside a Saudi special forces headquarters killing 15 people.          

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03059587, -0.03437759, -0.00743493, ..., -0.04873923,
          0.04763389, -0.01958877],
        [ 0.0177596 ,  0.043843  ,  0.04372307, ...,  0.03298516,
          0.01620394,  0.0029887 ],
        [-0.02677995,  0.0153729 , -0.01777462, ..., -0.0121145 ,
          0.01192207,  0.02817917],
        ...,
        [-0.03299621, -0.00457177, -0.01481386, ...,  0.03210658,
          0.00847826, -0.04025663],
        [ 0.01735225,  0.0332181 ,  0.01229005, ..., -0.00368242,
         -0.01222739,  0.04027026],
        [ 0.03095504,  0.04043927,  0.00885105, ..., -0.00964042,
         -0.00272356, -0.00579499]]], dtype=float32)>

In [68]:
# checkout a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03059587, -0.03437759, -0.00743493,  0.01186931, -0.00243629,
        -0.01053936,  0.0400699 ,  0.00449957,  0.02440733, -0.01782608,
         0.01874861, -0.04821157, -0.01975931, -0.04042726, -0.01419349,
        -0.01204587,  0.0329291 ,  0.01707755,  0.04039383,  0.00050868,
        -0.02068464,  0.017764  , -0.0437361 ,  0.04874834,  0.00471556,
        -0.00514314, -0.03385437, -0.01677211, -0.04696752, -0.04331082,
        -0.03077692, -0.01770347, -0.02619841,  0.01117283,  0.00107752,
         0.04559592, -0.03697809,  0.00460458, -0.00460855, -0.03516661,
         0.03038566, -0.04445101, -0.04675266, -0.0155939 , -0.04223281,
         0.02534885, -0.00990256, -0.02746801,  0.0246366 ,  0.00836175,
         0.00180513, -0.04565178, -0.00027264,  0.03552456,  0.02366177,
        -0.04905839,  0.03249303,  0.02950609,  0.02669755, -0.00216053,
        -0.03324278, -0.00292861, -0.01812459, -0.02150464,  0.02347476,
  

## modelling a text dataset

we start with a baseline and move on from there:
* Model 0 : Naive Bayes (Baseline)
* Model 1 : Feed-Forward neural network(Dense Model)
* Model 2 : LSTM model (RNN)
* Model 3 : GRU model(RNN)
* Model 4 : Bidirectional-LSTM model(RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : Tensorflow Hub Pretrained Feature Extractor Model (using transfer learning for NLP)
* Model 7 : same as model 6 with 10% of the training data

### Model 0 : Getting a Baseline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import 